# Avocado Price Prediction: Data Preparation

## CRISP-DM Phase 3: Data Preparation

In [44]:
import pandas as pd
# Load the dataset
df = pd.read_csv("../avocado2015to2022withweather.csv") 
# Display basic information
df.info()
# Display the first few rows
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45019 entries, 0 to 45018
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          45019 non-null  object 
 1   year_number   45019 non-null  int64  
 2   week_number   45019 non-null  int64  
 3   AveragePrice  45019 non-null  float64
 4   TotalVolume   45019 non-null  float64
 5   plu4046       45019 non-null  float64
 6   plu4225       45019 non-null  float64
 7   plu4770       45019 non-null  float64
 8   type          45019 non-null  object 
 9   region        45019 non-null  object 
 10  tavg          45019 non-null  float64
dtypes: float64(6), int64(2), object(3)
memory usage: 3.8+ MB


,Date,year_number,week_number,AveragePrice,TotalVolume,plu4046,plu4225,plu4770,type,region,tavg
0,2015-01-04,2015,2,0.74,1086363.97,612795.80,374420.68,9817.28,conventional,DallasFtWorth,3.4
1,2015-01-04,2015,2,1.12,17296.85,14569.66,1868.59,0.00,organic,PhoenixTucson,8.4
2,2015-01-04,2015,2,1.00,84612.39,26607.29,44341.92,3660.64,conventional,Spokane,-4.3
3,2015-01-04,2015,2,1.35,9895.96,4634.70,1647.92,0.00,organic,DallasFtWorth,3.4
4,2015-01-04,2015,2,0.65,1048062.16,770635.37,178418.32,6509.41,conventional,PhoenixTucson,8.4


In [45]:
#convert character to datetime
df["Date"] = pd.to_datetime(df["Date"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45019 entries, 0 to 45018
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          45019 non-null  datetime64[ns]
 1   year_number   45019 non-null  int64         
 2   week_number   45019 non-null  int64         
 3   AveragePrice  45019 non-null  float64       
 4   TotalVolume   45019 non-null  float64       
 5   plu4046       45019 non-null  float64       
 6   plu4225       45019 non-null  float64       
 7   plu4770       45019 non-null  float64       
 8   type          45019 non-null  object        
 9   region        45019 non-null  object        
 10  tavg          45019 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(2), object(2)
memory usage: 3.8+ MB


In [46]:
# check if there are any missing values
print(df.isnull().sum())

Date            0
year_number     0
week_number     0
AveragePrice    0
TotalVolume     0
plu4046         0
plu4225         0
plu4770         0
type            0
region          0
tavg            0
dtype: int64


In [47]:
# map (AKA factored) the type column to 0 and 1
df["type"] = df["type"].map({"conventional": 0, "organic": 1})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45019 entries, 0 to 45018
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          45019 non-null  datetime64[ns]
 1   year_number   45019 non-null  int64         
 2   week_number   45019 non-null  int64         
 3   AveragePrice  45019 non-null  float64       
 4   TotalVolume   45019 non-null  float64       
 5   plu4046       45019 non-null  float64       
 6   plu4225       45019 non-null  float64       
 7   plu4770       45019 non-null  float64       
 8   type          45019 non-null  int64         
 9   region        45019 non-null  object        
 10  tavg          45019 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(3), object(1)
memory usage: 3.8+ MB


In [48]:
# Turn the region (which is categorical) into binary columns (Dummy Variables))
df = pd.get_dummies(df, columns=["region"], drop_first=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45019 entries, 0 to 45018
Data columns (total 65 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Date                         45019 non-null  datetime64[ns]
 1   year_number                  45019 non-null  int64         
 2   week_number                  45019 non-null  int64         
 3   AveragePrice                 45019 non-null  float64       
 4   TotalVolume                  45019 non-null  float64       
 5   plu4046                      45019 non-null  float64       
 6   plu4225                      45019 non-null  float64       
 7   plu4770                      45019 non-null  float64       
 8   type                         45019 non-null  int64         
 9   tavg                         45019 non-null  float64       
 10  region_Atlanta               45019 non-null  bool          
 11  region_BaltimoreWashington   45019 non-nu

# Modeling

## 1- XGBoost Regressor

In [49]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Read and prepare the data
print("Loading data...")
df = pd.read_csv('../avocado2015to2022withweather.csv')

# Data quality check
print("\nChecking data quality...")
print("Missing values before cleaning:")
print(df.isnull().sum())

# Convert date to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Create additional time-based features
df['month'] = df['Date'].dt.month
df['day_of_week'] = df['Date'].dt.dayofweek
df['week_of_year'] = df['Date'].dt.isocalendar().week
df['year'] = df['Date'].dt.year

# Encode categorical variables
le_region = LabelEncoder()
le_type = LabelEncoder()
df['region_encoded'] = le_region.fit_transform(df['region'])
df['type_encoded'] = le_type.fit_transform(df['type'])

# Feature selection
features = [
    'region_encoded', 'type_encoded', 'year', 'month', 'day_of_week', 'week_of_year',
    'TotalVolume', 'tavg'
]

X = df[features]
y = df['AveragePrice']

# Print feature statistics
print("\nFeature Statistics:")
print(X.describe())

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a pipeline with imputer, scaler, and model
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('regressor', GradientBoostingRegressor(
        n_estimators=200,
        learning_rate=0.1,
        max_depth=5,
        min_samples_split=5,
        random_state=42
    ))
])

# Train the pipeline
print("\nTraining model pipeline...")
pipeline.fit(X_train, y_train)

# Model evaluation
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\n=== Model Performance ===")
print(f"Root Mean Squared Error: ${rmse:.3f}")
print(f"Mean Absolute Error: ${mae:.3f}")
print(f"R-squared Score: {r2:.3f}")

# Cross-validation
cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='r2')
print(f"\nCross-validation R² scores: {cv_scores}")
print(f"Average CV R² score: {cv_scores.mean():.3f} (+/- {cv_scores.std() * 2:.3f})")

# Feature importance visualization
feature_importance = pd.DataFrame({
    'feature': features,
    'importance': pipeline.named_steps['regressor'].feature_importances_
})
feature_importance = feature_importance.sort_values('importance', ascending=False)


def predict_price(date, region, type_val, temperature):
    """
    Predict avocado price based on date, region, type, and temperature.
    
    Parameters:
    date (str): Date in 'YYYY-MM-DD' format
    region (str): Region name
    type_val (str): 'conventional' or 'organic'
    temperature (float): Average temperature in Celsius
    
    Returns:
    float: Predicted price
    """
    try:
        # Convert date to datetime and extract features
        date = pd.to_datetime(date)
        year = date.year
        month = date.month
        day_of_week = date.dayofweek
        week_of_year = date.isocalendar().week
        
        # Encode categorical variables
        region_enc = le_region.transform([region])[0]
        type_enc = le_type.transform([type_val])[0]
        
        # Get average volume for the region
        region_data = df[df['region'] == region]
        if len(region_data) == 0:
            raise ValueError(f"Region '{region}' not found in training data")
            
        avg_volume = region_data['TotalVolume'].mean()
        
        # Create feature vector
        features = np.array([[
            region_enc, type_enc, year, month, day_of_week, week_of_year,
            avg_volume, temperature
        ]])
        
        # Make prediction using the pipeline
        predicted_price = pipeline.predict(features)[0]
        
        return predicted_price
        
    except Exception as e:
        print(f"Error during prediction: {str(e)}")
        return None

# Example usage
if __name__ == "__main__":
    # Example prediction
    test_date = '2023-01-15'
    test_region = 'StLouis'
    test_type = 'conventional'
    test_temperature = 9.3  # Example temperature in celsius
    
    predicted_price = predict_price(test_date, test_region, test_type, test_temperature)
    
    if predicted_price is not None:
        print(f"\nPrediction Example:")
        print(f"Date: {test_date}")
        print(f"Region: {test_region}")
        print(f"Type: {test_type}")
        print(f"Temperature: {test_temperature}°C")
        print(f"Predicted Price: ${predicted_price:.2f}")
        
        # Show historical comparison
        print("\nRecent historical prices for comparison:")
        recent_prices = df[
            (df['region'] == test_region) & 
            (df['type'] == test_type)
        ].sort_values('Date', ascending=False).head(5)
        
        for _, row in recent_prices.iterrows():
            print(f"Date: {row['Date'].strftime('%Y-%m-%d')}, "
                  f"Price: ${row['AveragePrice']:.2f}, "
                  f"Temperature: {row['tavg']:.1f}°C")

Loading data...

Checking data quality...
Missing values before cleaning:
Date            0
year_number     0
week_number     0
AveragePrice    0
TotalVolume     0
plu4046         0
plu4225         0
plu4770         0
type            0
region          0
tavg            0
dtype: int64

Feature Statistics:
       region_encoded  type_encoded          year         month   day_of_week  \
count    45019.000000  45019.000000  45019.000000  45019.000000  45019.000000   
mean        27.231436      0.499967   2018.574846      6.512850      5.925631   
std         16.026740      0.500006      2.272354      3.447458      0.663845   
min          0.000000      0.000000   2015.000000      1.000000      0.000000   
25%         13.000000      0.000000   2017.000000      4.000000      6.000000   
50%         27.000000      0.000000   2019.000000      7.000000      6.000000   
75%         41.000000      1.000000   2021.000000     10.000000      6.000000   
max         55.000000      1.000000   2022.000

c:\Users\ghara\Learning Hub\DataOps-Materials\Individual\project\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


## 2- Linear Regression

In [50]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

df = pd.read_csv("../avocado2015to2022withweather.csv") 
#convert character to datetime
df["Date"] = pd.to_datetime(df["Date"])
df["type"] = df["type"].map({"conventional": 0, "organic": 1})
df = pd.get_dummies(df, columns=["region"], drop_first=True)

# X is feature and y is target
X = df.drop(columns=["Date", "AveragePrice","plu4046","plu4225","plu4770"])  # remove the target column and the date column
y = df["AveragePrice"]  

In [51]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train)

# Initialize linear regression model
lin_reg = LinearRegression()

# Train the model
lin_reg.fit(X_train, y_train)

# Predict the target values
y_pred = lin_reg.predict(X_test)

# Evaluate the model
print("Linear Regression R² Score:", lin_reg.score(X_test, y_test))

       year_number  week_number  TotalVolume  type  tavg  region_Atlanta  \
27012         2019           48    340758.38     0   2.6           False   
16927         2018           10   1715963.11     0  11.9           False   
8313          2016           31      5882.29     1  20.3           False   
28165         2020            6     15288.82     1   8.7           False   
6324          2016           12      4036.93     1   3.7           False   
...            ...          ...          ...   ...   ...             ...   
11284         2017            7     22901.55     1   3.9           False   
44732         2022           51    671179.99     0  11.7            True   
38158         2021           43     10274.23     1  13.1           False   
860           2015           10     16227.56     1   4.7            True   
15795         2017           51      8486.81     1  23.6           False   

       region_BaltimoreWashington  region_BirminghamMontgomery  region_Boise  \
27012  

## 3- Random forest

In [52]:
from sklearn.ensemble import RandomForestRegressor

df = pd.read_csv("../avocado2015to2022withweather.csv") 
#convert character to datetime
df["Date"] = pd.to_datetime(df["Date"])
df["type"] = df["type"].map({"conventional": 0, "organic": 1})
df = pd.get_dummies(df, columns=["region"], drop_first=True)

# Initialize Random Forest model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

## Predicting the price of Avocado 

In [53]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Predictions on test data
y_pred = model.predict(X_test)
# Evaluation metrics
print("R² Score:", r2_score(y_test, y_pred))
print("Mean Absolute Error (MAE):", mean_absolute_error(y_test, y_pred))
print("Root Mean Squared Error (RMSE):", np.sqrt(mean_squared_error(y_test, y_pred)))

R² Score: 0.9230294951148925
Mean Absolute Error (MAE): 0.07674275036464459
Root Mean Squared Error (RMSE): 0.10896028362777821


## Evaluation of forecasting price of Avocado with random forest

In [54]:
# Load real the dataset for 2023
X_test_2023 = pd.read_csv("../avocadoafter2023withweather.csv") 

X_test_2023["Date"] = pd.to_datetime(df["Date"])
X_test_2023["type"] = X_test_2023["type"].map({"conventional": 0, "organic": 1})
X_test_2023 = pd.get_dummies(X_test_2023, columns=["region"], drop_first=True)

# Check for missing columns
print("Columns in X_test_2023 (before modification):", X_test_2023.columns)

# Add missing columns with default values
missing_columns = [col for col in X_test.columns if col not in X_test_2023.columns]

for col in missing_columns:
    X_test_2023[col] = False  # Default value for binary columns

# Sort columns to match the order in X_test
X_test_2023 = X_test_2023[X_test.columns]

# predict the target values
y_pred_2023 = model.predict(X_test_2023)

# Add the predicted values to the dataset
X_test_2023["Predicted"] = y_pred_2023

# Save the dataset
X_test_2023.to_csv("../forcastingReport/forcasting_2023.csv", index=False)

Columns in X_test_2023 (before modification): Index(['Date', 'year_number', 'week_number', 'AveragePrice', 'TotalVolume',
       'plu4046', 'plu4225', 'plu4770', 'type', 'tavg', 'region_Atlanta',
       'region_BaltimoreWashington', 'region_BirminghamMontgomery',
       'region_Boise', 'region_Boston', 'region_BuffaloRochester',
       'region_California', 'region_Charlotte', 'region_CincinnatiDayton',
       'region_Columbus', 'region_DallasFtWorth', 'region_Denver',
       'region_Detroit', 'region_GrandRapids', 'region_GreatLakes',
       'region_HarrisburgScranton', 'region_HartfordSpringfield',
       'region_Houston', 'region_Indianapolis', 'region_Jacksonville',
       'region_LasVegas', 'region_LosAngeles', 'region_Louisville',
       'region_Midsouth', 'region_Nashville', 'region_NewOrleans',
       'region_NewYork', 'region_NorthernNewEngland', 'region_Orlando',
       'region_PeoriaSpringfield', 'region_Philadelphia',
       'region_PhoenixTucson', 'region_Pittsburgh', 'regi